This notebook presents METIS LSS L-band simulation of three models of a young stellar object (YSO).

In [ ]:
import scopesim as sim
sim.bug_report()

# Edit this path if you have a custom install directory, otherwise comment it out. [For ReadTheDocs only]
sim.rc.__config__["!SIM.file.local_packages_path"] = "../../../"

In [ ]:
import numpy as np

from astropy.io import fits
from astropy import units as u

from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline

If you haven't got the instrument packages yet, uncomment the following cell, which will install the packages into `./inst_pkgs`, a subdirectory of your current working directory. This is the default location where scopesim looks for instrument packages.

In [ ]:
# sim.download_packages(["METIS", "ELT", "Armazones"])

If you have downloaded the packages but to a different location, you can set
```python
sim.rc.__config__["!SIM.file.local_packages_path"] = <"/path/to/inst/pkgs">
```
We recommend, however, to create a working directory for each simulation project and to use the default installation of packages into a subdirectory. Keeping simulation results and configuration files together that way makes it easy to reconstruct later the exact conditions under which a simulation was run.

# Preparation of source cubes

The input data are cubes of three different models of the same YSO, HD100546. We keep the names of FITS files, the `Source` objects and the results of the Scopesim simulations in dictionaries, indexed by short names for the models.

In [ ]:
fitsfiles = {}
fitsfiles['cav'] = "models_Lband_HD100546_cav_f100PAH.cube_3.0mas.fits"
fitsfiles['emptycav'] = "models_Lband_HD100546_empytcav.cube_3.0mas.fits"
fitsfiles['gap'] = "models_Lband_HD100546_gap100.cube_3.0mas.fits"
models = list(fitsfiles.keys())
print("Model names:", models)

The FITS files can be downloaded from the Scopesim server. If you already have them, make sure that the files are in the current working directory.

In [ ]:
fitsfiles = dict(zip(fitsfiles, sim.download_example_data(*fitsfiles.values())))

The file headers are not yet in the form that Scopesim understands and we make two minor modifications: 
- Set CRVAL to 0, because Scopesim cannot look elsewhere
- Set BUNIT keyword (the files have the keyword UNITS, which is non-standard)
- The cubes contain some negative values. We replace these with 0.
- We introduce a factor `scale_delt` to increase the pixel size, which makes features more visible. If you want to simulate the original source pixel scale, set `scale_delt` to 1.

In [ ]:
sources = {}
scale_cdelt = 5.
for model, fitsfile in fitsfiles.items():
    with fits.open(fitsfile) as hdul:
        hdul[0].header['CRVAL1'] = 0.
        hdul[0].header['CRVAL2'] = 0.
        hdul[0].header['CDELT1'] *= scale_cdelt
        hdul[0].header['CDELT2'] *= scale_cdelt
        hdul[0].header['BUNIT'] = hdul[0].header['UNITS']
        hdul[0].data[hdul[0].data < 0] = 0
        sources[model] = sim.Source(cube=hdul)

To get an impression of what the data look like we display the cubes summed along the wavelength direction.

In [ ]:
# Determine plot limits in arcsec from header keywords (these are in degrees)
hdr = sources['cav'].cube_fields[0].header
i_lim = np.array([0, hdr['NAXIS1']])
x_lim = hdr['CRVAL1'] + hdr['CDELT1'] * (i_lim + 1 - hdr['CRPIX1']) * 3600
j_lim = np.array([0, hdr['NAXIS2']])
y_lim = hdr['CRVAL2'] + hdr['CDELT2'] * (j_lim + 1 - hdr['CRPIX2']) * 3600

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
for i, (model, src) in enumerate(sources.items()):
    im = axes[i].imshow(src.cube_fields[0].data.sum(axis=0) + 1e-14,   # add small positive value to avoid 0 in LogNorm
                        origin='lower', norm=LogNorm(vmin=1e-4, vmax=10),
                        extent=(x_lim[0], x_lim[-1], y_lim[0], y_lim[-1]))
    axes[i].set_xlabel("arcsec")
    axes[i].set_ylabel("arcsec")
    axes[i].set_title(model);
fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.9, 0.12, 0.02, 0.75])
fig.colorbar(im, cax=cbar_ax);

In wavelength, the cubes are sampled on a linear wavelength grid from 3.1 to 4 $\mu\mathrm{m}$, with a step size of $0.2\,\mu\mathrm{m}$:

In [ ]:
sources['cav'].cube_fields[0].wave

Note that the source object does not cover the entire spatial and wavelength range permitted by METIS: The spatial extent is about 2.5 arcsec, compared to the METIS slit length of 8 arcsec. The wavelength range is 3.1 to 4 $\mu$m, whereas METIS permits 2.9 to 4.2 $\mu$m. This will be visible in the simulated raw and rectified spectra below.

# Simulation with Scopesim

The cubes are observed in the long-slit spectroscopic mode in the L band. As usual, there are four steps: `UserCommands` -> `OpticalTrain` -> `observe` -> `readout` to arrive at a detector image. The optical train can be reused for observation of different sources when `update=True` is set in `observe()`. 

The simulation uses the `AutoExposure` effect to break down the requested exposure time into `NDIT` subexposures of integration time `DIT` so as to prevent saturation. For this to work, the parameters `dit` and `ndit` currently have to be set to `None` explicitly (here and in every `readout` command below). This will hopefully change in future versions of Scopesim.

In [ ]:
exptime = 3600.   # seconds
cmd = sim.UserCommands(use_instrument="METIS", set_modes=["lss_l"],
                       properties={"!OBS.exptime": exptime, "!OBS.dit": None, "!OBS.ndit": None})

In [ ]:
metis = sim.OpticalTrain(cmd)

In [ ]:
results = {}
for model, src in sources.items():
    print(f'Observing model "{model}..."')
    metis.observe(src, update=True)
    results[model] = metis.readout(detector_readout_mode='auto', dit=None, ndit=None)[0]
    print("-----")

In [ ]:
plt.figure(figsize=(15, 5))
for i, (model, result) in enumerate(results.items()):
    plt.subplot(1, 3, i+1)
    plt.imshow(result[1].data, origin='lower', norm=LogNorm())
    plt.xlabel("pixel")
    plt.ylabel("pixel")
    plt.title(model);

The following is a simulation of a blank-sky observation, which will be used for background subtraction. Given that the source is fairly compact, one could estimate the background from the science observation as well.

In [ ]:
sky = sim.source.source_templates.empty_sky()
metis.observe(sky, update=True)
bgresult = metis.readout(detector_readout_mode="auto", dit=None, ndit=None)[0]

We subtract the background from the data before rectifying the spectra. Here's where the quantization effect would have caused problems: background subtraction from noisy data leads to negative values, which cannot be represented by unsigned integers and would wrap around to large positive values (in `uint16`: $1 - 2 = 2^{16} - 1 = 65535$). Therefore the data would have to be converted to `float` before proceeding further.

In [ ]:
for (_, result) in results.items():
    result[1].data -= bgresult[1].data

We now use the `rectify_traces` method to resample the spectra to a linear grid in wavelength and spatial position. The method is attached to the `SpectraTraceList` effect, which holds the information on the geometrical mapping onto the detector. Note that this is an optimistic approach as it uses the same transformations used before to simulate the detector data. When reducing real data, the transformation parameters would have to be estimated from dedicated calibration measurements and would therefore have a degree of uncertainty.

In [ ]:
rectified = {}
tracelist = metis['spectral_traces']
for i, (model, result) in enumerate(results.items()):
    rectified[model] = tracelist.rectify_traces(result, -4.0, 4.0)

Before showing the rectified spectra, we convert the pixel numbers to wavelength and spatial position along the slit. For this purpose we use the WCS keywords in the headers of the rectified HDUs. 

In [ ]:
from astropy.wcs import WCS
hdr = rectified['cav'][1].header
wcs = WCS(hdr)
naxis1, naxis2 = hdr['NAXIS1'], hdr['NAXIS2']
det_xi = wcs.all_pix2world(1, np.arange(naxis2), 0)[1] * u.Unit(wcs.wcs.cunit[1])
det_xi = det_xi.to(u.arcsec)

det_wave = wcs.all_pix2world(np.arange(naxis1), 1, 0)[0] * u.Unit(wcs.wcs.cunit[0])
det_wave = det_wave.to(u.um)          

In [ ]:
plt.figure(figsize=(15, 15))
for i, (model, result) in enumerate(rectified.items()):
    plt.subplot(3, 1, i+1)
    plt.imshow(result[1].data, origin='lower', norm=LogNorm(),
               extent=(det_wave[0].value, det_wave[-1].value, det_xi[0].value, det_xi[-1].value), aspect='auto')
    plt.ylabel(r"position along slit [arcsec]")
    plt.xlabel(r"Wavelength [$\mu$m]")
    plt.title(model);
    plt.colorbar()

As stated above the input source object does not fill the entire spatial and wavelength ranges covered by METIS. In a real observation the entire frame shown here would be filled.

The results have to be saved to disk explicitely so they can be analysed with external tools. 

In [ ]:
from pathlib import Path

# Change to True if you want to save the rectified data
save_rectified = False
if save_rectified:
    for i, (model, result) in enumerate(rectified.items()):
        outfile = Path(fitsfiles[model]).stem + "-simulated_LSS_L" + Path(fitsfiles[model]).suffix
        result.writeto(outfile, overwrite=True)
        print(fitsfiles[model], "--->", outfile)
    bgresult.writeto("models_Lband_HD100546-background_simulated_LSS_L.fits", overwrite=True)

# Rotating the field
In Scopesim, the METIS slit is always aligned with the x-axis. To simulate different slit orientations, the input cube has to be rotated. We define a function to do this (for more explanation of the steps see the notebook `LSS_AGN-01_preparation.ipynb`).

In [ ]:
from astropy.wcs import WCS
from scipy.interpolate import RectBivariateSpline

In [ ]:
def rotate_cube(incube, angle):
    """Rotate input cube by angle
    
    Parameters
    ----------
    incube : ImageHDU
    angle : float
        angle in degrees, counterclockwise from positive x-axis
    """
    rangle = angle * np.pi / 180     # degrees to radians
    
    wcs_orig = WCS(incube.header).sub(2)
    wcs_orig.wcs.ctype = ["LINEAR", "LINEAR"]   # avoids discontinuity around RA=0 degrees
    wcs_rot = WCS(incube.header).sub(2)
    wcs_rot.wcs.ctype = ["LINEAR", "LINEAR"]
    
    wcs_rot.wcs.pc = [[np.cos(rangle), np.sin(rangle)],
                     [-np.sin(rangle), np.cos(rangle)]]
    i_orig = np.arange(incube.header['NAXIS1'])
    j_orig = np.arange(incube.header['NAXIS2'])
    x_orig, y_orig = wcs_orig.all_pix2world(i_orig, j_orig, 0)
    x_orig[x_orig >180] -= 360.     # RA continuous across 0
    i_rot, j_rot = np.meshgrid(i_orig, j_orig)
    x_rot, y_rot = wcs_rot.all_pix2world(i_rot, j_rot, 0)
    
    for iplane, plane in enumerate(incube.data):
        interp = RectBivariateSpline(y_orig, x_orig, plane, kx=1, ky=1)
        incube.data[iplane,] = interp(y_rot, x_rot, grid=False)
    incube.header['ANGLE'] = angle, "slit rotation [deg]"
    return incube

The function is applied to the data cubes before creating the `Source` object. We choose an angle of 45 degrees. The simulation then proceeds as above.

In [ ]:
angle = 45
rotsources = {}
for model, fitsfile in fitsfiles.items():
    with fits.open(fitsfile) as hdul:     
        hdul[0].header['CRVAL1'] = 0.
        hdul[0].header['CRVAL2'] = 0.
        hdul[0].header['CDELT1'] *= scale_cdelt
        hdul[0].header['CDELT2'] *= scale_cdelt
        hdul[0].header['BUNIT'] = hdul[0].header['UNITS']
        hdul[0].data[hdul[0].data < 0] = 0
        hdul[0] = rotate_cube(hdul[0], angle)
        rotsources[model] = sim.Source(cube=hdul)

In [ ]:
rotresults = {}
for model, src in rotsources.items():
    print(f'Observing model "{model}"...')
    metis.observe(src, update=True)
    rotresults[model] = metis.readout(detector_readout_mode='auto', dit=None, ndit=None)[0]
    print("-----")

In [ ]:
rotrectified = {}
for model, result in rotresults.items():
    result[1].data -= bgresult[1].data
    rotrectified[model] = tracelist.rectify_traces(result, -4.0, 4.0)

In [ ]:
plt.figure(figsize=(15, 15))
for i, (model, result) in enumerate(rotrectified.items()):
    plt.subplot(3, 1, i+1)
    plt.imshow(result[1].data, origin='lower', norm=LogNorm(),
               extent=(det_wave[0].value, det_wave[-1].value, det_xi[0].value, det_xi[-1].value), aspect='auto')
    plt.ylabel(r"position along slit [arcsec]")
    plt.xlabel(r"Wavelength [$\mu$m]")
    plt.title(model);
    plt.colorbar()

In [ ]:
# Change to True if you want to save the rectified data
save_rotrectified = False
if (save_rotrectified):
    for i, (model, result) in enumerate(rotresults.items()):
        outfile = Path(fitsfiles[model]).stem + "-rot_" + str(angle) + "-simulated_LSS_L" + Path(fitsfiles[model]).suffix
        result.writeto(outfile, overwrite=True)
        print(fitsfiles[model], "--->", outfile)

We compare the rotated spectrum for the `emptycav` model to the spectrum without rotation.

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(6, 12))

ax1.plot(det_xi, rectified['emptycav'][1].data[:, 1500], label="no rotation")
ax1.plot(det_xi, rotrectified['emptycav'][1].data[:, 1500], label=f"rotation {angle} deg")
ax1.set_xlim(x_lim[0], x_lim[-1])
ax1.set_ylim(2e4, 1e9)
ax1.set_xlabel("arcsec")
ax1.semilogy()
ax1.legend()

fig.subplots_adjust(left=0.1)
ax2.imshow(rotsources['emptycav'].cube_fields[0].data.sum(axis=0) + 1e-14, norm=LogNorm(vmin=1e-4, vmax=10),
           extent=(x_lim[0], x_lim[-1], y_lim[0], y_lim[-1]))
ax2.set_xlabel("arcsec")
ax2.set_ylabel("arcsec")

# Flux calibration

The pixel values in the detector images give electrons accumulated over the exposure time. To get back to physical units, e.g. Jy, one has to perform a flux calibration. As in real observations, we do this here with the observation of a standard star. For simplicity, we use a star with a spectrum that is constant at $f_{\nu} = 1\,\mathrm{Jy}$. We observe it with the identical setup as the science targets above, except that the exposure time is reduced to 1 second.

In [ ]:
std = sim.source.source_templates.star(flux=1 * u.Jy)

In [ ]:
metis.observe(std, update=True)

In [ ]:
std_exptime = 1  # second
std_result = metis.readout(exptime=std_exptime, detector_readout_mode='auto', dit=None, ndit=None)[0]

We subtract the background (scaled to the exposure time of the standard exposure) and rectify.

In [ ]:
std_result[1].data -= bgresult[1].data / exptime
std_rectified = tracelist.rectify_traces(std_result, -4, 4)

In [ ]:
plt.figure(figsize=(15, 7))
plt.imshow(std_rectified[1].data, origin='lower', norm=LogNorm(),
           extent=(det_wave[0].value, det_wave[-1].value, det_xi[0].value, det_xi[-1].value), aspect='auto')
plt.ylabel(r"position along slit [arcsec]")
plt.xlabel(r"Wavelength [$\mu$m]")
plt.title("Standard star");
plt.colorbar();

In [ ]:
xmin, xmax = 675, 825
xaxis = np.arange(xmin, xmax)
y_1, y_2 = 200, 1750
lam_1, lam_2 = det_wave[y_1], det_wave[y_2]

plt.figure(figsize=(10, 5))
plt.plot(xaxis, std_rectified[1].data[xmin:xmax, y_1], label=fr"$\lambda = {lam_1.value:.2f}\,\mu\mathrm{{m}}$")
plt.plot(xaxis, std_rectified[1].data[xmin:xmax, y_2], label=fr"$\lambda = {lam_2.value:.2f}\,\mu\mathrm{{m}}$")
plt.plot(xaxis, std_rectified[1].data[xmin:xmax, :].mean(axis=1), label='average')
plt.vlines((732 - 10, 732 + 10), 1, 1e5, colors='black', linestyles='dashed', label='extraction aperture')
plt.xlabel("pixel")
plt.ylabel("e/s")
plt.semilogy()
plt.ylim(10, 4e6)
plt.legend()
plt.title("spatial cuts through standard spectrum");

The total flux density of 1 Jy from the star is spread over the point spread function, which extends far from the central position. It is cut by the finite aperture of the spectroscopic slit, and will be cut further by the finite extent of the extraction aperture over which we will sum the two-dimensional spectrum. Most of the flux is contained in the core of the PSF; from the figure, we can take an aperture size in the spatial direction of 20 pixels - outside this aperture, the spectrum is dominated by noise.  
The slit that we used is

In [ ]:
print(metis['slit_wheel'].current_slit)
# TODO: This used to return "C-38_1". The headers are made ESO
#       Compliant, and not all headers have been replaced.
#       See https://github.com/AstarVienna/irdb/pull/146
#std_result[0].header

with a width of 38.1 mas, corresponding to 7 pixels. As this notebook focuses on the simulator rather than the data reduction, we neglect slit and aperture losses and assume that the signal integrated over 20 pixels in the spatial direction corresponds to the input flux density of 1 Jy. The error is on the order of 2 per cent. 

In [ ]:
hwidth = 10     # pixels, half width of extraction aperture
std_1d = std_rectified[1].data[731-hwidth:731+hwidth].sum(axis=0)

In [ ]:
plt.plot(det_wave, std_1d)
plt.xlabel('Wavelength (um)')
exptime = std_rectified[0].header['EXPTIME']
plt.ylabel('electrons/s')
plt.title(r"Extracted standard spectrum ($T_{\mathrm{exp}} = 1\,\mathrm{s}$)");

We construct the flux conversion curve that takes the observed spectrum to the calibrated spectrum in Jy. Note that there are strong excursions at the edge of the spectroscopic filter and at deep atmospheric absorption features. The object flux at these wavelengths is not recoverable.

In [ ]:
flux_conv = 1 * u.Jy / (std_1d * u.electron)
plt.plot(det_wave, flux_conv)
plt.ylim(0, 1e-5)
plt.xlabel("Wavelength (um)")
plt.ylabel("Jy / (e/s)")
plt.title("Flux conversion");

We apply the flux calibration to the two-dimensional spectra from above, taking into account the different exposure time, and display them. Note how the PAH feature at around 3.3 um now becomes apparent in the disk.

In [ ]:
emptycav_Jy     = flux_conv * rectified['emptycav'][1].data / exptime
emptycav_Jy_rot = flux_conv * rotrectified['emptycav'][1].data / exptime

In [ ]:
plt.imshow(emptycav_Jy.value)

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(13, 12))
ax1.imshow(emptycav_Jy.value, origin='lower', 
           norm=LogNorm(vmin=1e-5, vmax=1), 
           extent=(det_wave[0].value, det_wave[-1].value, det_xi[0].value, det_xi[-1].value), aspect='auto')
ax1.set_ylabel(r"position along slit [arcsec]")
ax1.set_xlabel(r"Wavelength [$\mu$m]") 
ax1.set_title(model + ", no rotation")

img = ax2.imshow(emptycav_Jy_rot.value, origin='lower',
                 norm=LogNorm(vmin=1e-5, vmax=1), 
                 extent=(det_wave[0].value, det_wave[-1].value, det_xi[0].value, det_xi[-1].value), aspect='auto')
ax2.set_ylabel(r"position along slit [arcsec]")
ax2.set_xlabel(r"Wavelength [$\mu$m]") 
ax2.set_title(model + ", rotated by " + str(angle) + " degrees");

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(img, cax=cbar_ax, label='Jy')

Finally, we perform background subtraction and flux calibration on all simulations. The data are now normalised to an exposure time of 1 second and therefore have units 'electrons per second'.

In [ ]:
# Change to True if you want to save the calibrated data
save_calibrated = False
for i, (model, result) in enumerate(rectified.items()):
    result[1].data = flux_conv.value * result[1].data / exptime
    result[1].header['BUNIT'] = "e/s"
    if save_calibrated:
        outfile = (Path(fitsfiles[model]).stem + "-simulated_LSS_L-bgsub_fluxcal" 
                   + Path(fitsfiles[model]).suffix)
        result.writeto(outfile, overwrite=True)
        print("--->", outfile)

for i, (model, result) in enumerate(rotrectified.items()):
    result[1].data = flux_conv.value * result[1].data / exptime
    result[1].header['BUNIT'] = "e/s"
    if save_calibrated:
        outfile = (Path(fitsfiles[model]).stem + "-rot_" + str(angle) 
                   + "-simulated_LSS_L-bgsub_fluxcal" + Path(fitsfiles[model]).suffix)
        result.writeto(outfile, overwrite=True)
        print("--->", outfile)

# Comparison to input spectrum

In [ ]:
result_extract = rectified['emptycav'][1].data[731-hwidth:731+hwidth].sum(axis=0)

In [ ]:
src_extract = sources['emptycav'].cube_fields[0].data[:, 88:91, 87:92].sum(axis=(1, 2))
src_wave = sources['emptycav'].cube_fields[0].wave

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(det_wave, result_extract, label="flux-calibrated simulated spectrum")
# The scaling for the source spectrum has been estimated by eye
plt.plot(src_wave, src_extract * 1500, label="input spectrum")
plt.xlabel(r"Wavelength [$\mu$m]")
plt.ylabel("Relative flux")
plt.legend()
plt.xlim(3.1, 4.0);